In [2]:
import csv
import pandas as pd
import numpy as np

In [3]:
bids = pd.read_csv("BidData20160106.csv")
bids.head()

,Tags,Bid ranges (gross),Bids,Impressions won,Winning bid CPM (gross),Close CPM,Win %
0,(No tag),0.2 - 0.3,2,0,0.00,0.00,0%
1,(No tag),0 - 0.1,33,1,0.05,0.04,3.03%
2,(No tag),0.1 - 0.2,4,1,0.12,0.06,25%
3,(No tag),0.3 - 0.4,5,3,0.34,0.20,60%
4,300x250_d_image_only_atf,29.5 - 29.6,1,1,29.54,0.35,100%


In [4]:
tag_array = ['fb_chr_tags','fb_intl_tags','fb_us_tags']

fb_us_tags = ['gut_933_1','gut_933_2','gut_933_3','gut_933_4']
fb_intl_tags = ['gut_778_1','gut_778_2','gut_778_3','gut_778_4']
fb_chr_tags = ['gut_742_1','gut_742_2','gut_742_3','gut_742_4']

floors = np.arange(0.00, 6.00, 0.05).tolist()


In [5]:
tags = bids['Tags']
    
filtered_us = tags.isin(fb_us_tags)
filtered_us_chr = tags.isin(fb_chr_tags)
filtered_intl = tags.isin(fb_intl_tags)

us_bids = bids[filtered_us]

chr_bids = bids[filtered_us_chr]
intl_bids = bids[filtered_intl]
buckets = ['us_bids','chr_bids','intl_bids']

In [6]:
def originalRev(x):
   return x['Close CPM']*(x['Impressions won']/1000)

                
def calculateRev(impressionsWon, winCPM, closeCPM, floor):
    if floor < closeCPM and floor < winCPM:
        revenue = winCPM*(impressionsWon/1000)
        return revenue
    elif winCPM < floor:
        revenue = 0.0
    elif closeCPM < floor and floor < winCPM:
        revenue = (floor + 0.01)*(impressionsWon/1000)
    else:
        revenue = 0.0
    return revenue

'''This function uses calculateRev and originalRev to calculate revenues at hypothetical floors at each bid range.'''
def revColumn(floors,bidData):
    revenue_at_floor = {'originalRevenue':[]}
    revenue_at_floor['originalRevenue'] = originalRev(bidData).tolist()
    for floor in floors:
        revenue_at_floor[floor] = []
        for index, row in bidData.iterrows():
            rev = calculateRev(row['Impressions won'],row['Winning bid CPM (gross)'], row['Close CPM'], floor)
            revenue_at_floor[floor].append(rev)
    rev_by_bid_range = pd.DataFrame(revenue_at_floor)
    tags = bidData['Tags'].reset_index()['Tags']
    rev_by_bid_range = rev_by_bid_range.join(tags)
    headers = list(rev_by_bid_range.columns.values)
    headers.insert(0, headers.pop())
    return rev_by_bid_range[headers]

def flooringRecommendations(revenue_df):
    return revenue_df.groupby('Tags')
                                
floors = np.arange(0.00, 6.00, 0.05).tolist()

In [28]:
a = revColumn(floors,us_bids)


In [51]:
b = pd.pivot_table(a,index=["Tags"])
print b

                0.0      0.05       0.1      0.15       0.2      0.25  \
Tags                                                                    
gut_933_1  0.007465  0.007465  0.007465  0.007465  0.007465  0.007465   
gut_933_2  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
gut_933_3  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
gut_933_4  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

                0.3     0.35       0.4      0.45       ...         5.55  5.6  \
Tags                                                   ...                     
gut_933_1  0.007465  0.00507  0.005775  0.006479       ...            0    0   
gut_933_2  0.000000  0.00000  0.000000  0.000000       ...            0    0   
gut_933_3  0.000000  0.00000  0.000000  0.000000       ...            0    0   
gut_933_4  0.000000  0.00000  0.000000  0.000000       ...            0    0   

           5.65  5.7  5.75  5.8  5.85  5.9  5.95  originalRevenue  
Tags        

In [45]:
c = {"q":[1,2,3]}
max(c["q"])

3

[1]
[1, 2]
